# Modelo de recomendaciones para vacantes de trabajo

La idea de esta prueba técnica es poder construir un modelo sencillo para poder recomendar vacantes laborales a personas en búsqueda de empleo. 

Para ello vamos a trabajar la prueba en dos fases:
1. Extracción y preparación de los datos que vayan a ser usados como input 
2. Construcción y ejecución del modelo que se proponga

## Fase 1: Extraccion y preparación de los datos

En esta primera parte se requiere construir la base de datos que se usará para generar recomendaciones laborales. Para ello vamos a considerar dos fuentes de datos: 

1. Datos de personas buscando empleo
2. Datos de vacantes de trabajo disponibles

### Datos de personas en búsqueda de empleo

En este primer bloque de código debes traer los datos de las personas en busqueda laboral 

url = https://docs.google.com/spreadsheets/d/1O1N08D-5NnWjnKJ6LrU0hjbYjgN6BPrN1Cg1FWQ8Y4w/edit?usp=sharing

**diccionario de columnas:**

* `anonymous_user` : identificador de usuario anonimizado
* `puestos_postulandose` : array() de los tipos de puestos a los que la persona desea aplicar
* `subareas_trabajo` : array() de las areas de trabajo a las que la persona desea aplicar
* `nivel_de_ingles` : nivel de ingles de la persona  

Sigue la documentacion aquí como una posible guía para traer los datos desde el google sheets: 

[Como traer datos a pandas desde google sheets](https://towardsdatascience.com/read-data-from-google-sheets-into-pandas-without-the-google-sheets-api-5c468536550)

al final asegurate de que el dataframe conteniendo la data de la personas se guarde como `personas_df`

In [ ]:
import pandas as pd

import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os
import numpy as np

sheet_id="1O1N08D-5NnWjnKJ6LrU0hjbYjgN6BPrN1Cg1FWQ8Y4w"
sheet_name="raw_data"
base = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

personas_df=pd.read_csv(base)

# elimino los registros que no tienen informaciòn ni en el puesto postulante ni en las subareas de trabajo

personas_df=personas_df.dropna(subset=["puestos_postulandose","subareas_trabajo"], how='all')

# ver si existes ususarios repetidos
print(personas_df['anonymous_user'].duplicated().any())

### Datos de las vacantes de trabajo disponibles

En este segundo bloque de codigo debes implementar un proceso que traiga vacantes publicas disponibles de alguna fuente de datos relevante, e.j. Linkedin. El proceso sugerido es: 

1. Definir fuente de datos y viabilidad de la extraccion de datos con Web-scrapping
2. Definir atributos relevantes de las vacantes a extraer para armar la estructura del dataframe
3. Asegurate de guardar todos los datos de vacantes en un dataframe `vacantes_df`

**Ten en cuenta, que los atributos a escoger de las vacantes deben de alguna manera permitirte hacer match con la data de las personas que te fue otorgada en el paso anterior**.

Es important anotar que solo es preciso que construyas un flujo que te permita guardar los datos en un dataframe, NO es necesario montar una base de datos SQL ni nada similar.

Aquí te dejamos algunos ejemplos de procesos de extraccion de vacantes de empleo de algunos portales: 

* [Usando Selenium o Scrapy](https://towardsdatascience.com/when-job-hunting-meets-data-science-part-1-e8f64867d8c)
* [Otro caso de uso con Scrapy](https://towardsdatascience.com/automate-your-job-search-with-python-and-github-actions-1dc818844c0)
* [Automatic job search with python](https://towardsdatascience.com/automating-my-job-search-with-python-ee2b465c6a8f)

In [ ]:
# funciones para realizar la busqueda de los empleos

## Buscador del trabajo se realizò en linkedin

def buscador(job_title,location="colombia"):
    temp ="https://co.linkedin.com/jobs/search?keywords={}&location={}&geoId=102361989&trk=public_jobs_jobs-search-bar_search-submit"
    url = temp.format(job_title,location)
    return url

## extraciòn de la informacion relevante
## posicion: trabajo que busca
## empresa. empresa que publica el trabajo
## caracteristicas del trabajo

def info_trabajo(url):
    request=requests.get(url)
    soup= BeautifulSoup(request.text,"html.parser")
    cards= soup.find_all('a','base-card__full-link')
    posicion,empresa,caracteristicas=[],[],[]
    for i in range(len(cards)):
        posicion.append(soup.find_all('div', {'class' : 'base-search-card__info'})[i].h3.text.strip())
        empresa.append(soup.find_all('div', {'class' : 'base-search-card__info'})[i].h4.text.strip())
        caracteristicas=soup.find_all('div', {'class' : 'base-search-card__info'})[i].p.text.strip()
    matriz_trabajo=pd.DataFrame()
    matriz_trabajo["trabajo"]=posicion
    matriz_trabajo["empresa"]=empresa
    matriz_trabajo["caracteristicas"]=caracteristicas
    return matriz_trabajo

# creo un data frame para realizar las busquedas de trabajo
busquedas=[]
usuario=[]
ingles=[]
for n,m in zip(personas_df.anonymous_user,personas_df.nivel_de_ingles):
    for i in personas_df.puestos_postulandose:
        try:
                jobs=i
                jobs=jobs.replace("]"," ")
                jobs=jobs.replace("["," ")
                jobs=jobs.split(",")

                for t in jobs:
                    busquedas.append(t)
                    usuario.append(n)
                    ingles.append(m)

        except:
            busquedas.append("no trabajo") 
            usuario.append(n)
            ingles.append(m)
busqueda=pd.concat([pd.DataFrame(busquedas,columns=["busqueda"]),pd.DataFrame(ingles,columns=["ingles"]),
                    pd.DataFrame(usuario,columns=["usuario"])],axis=1)

# Uso también el arry de subsector, por lo que se puede ver tiene palabras claves para realizar busquedas de trabajo 

sub_areas=[]
usuario=[]
ingles=[]
for n,m in zip(personas_df.anonymous_user,personas_df.nivel_de_ingles):
    for j in personas_df.subareas_trabajo:
        try:
                sub=j
                sub=sub.replace("["," ")
                sub=sub.replace("]"," ") 
                sub=sub.split(",")
                for t in sub:
                    sub_areas.append(t)
                    usuario.append(n)
                    ingles.append(m)
        except:
            sub_areas.append("no trabajo") 
            usuario.append(n)
            ingles.append(m)
sub_area=pd.concat([pd.DataFrame(sub_areas,columns=["busqueda"]),pd.DataFrame(ingles,columns=["ingles"]),
                    pd.DataFrame(usuario,columns=["usuario"])],axis=1) 

# genero listas con valores unicos de cada área de trabajo y sub sector, pensando que existen muchas personas que buscan el mismo 
# empleo y por lo tanto no es necesario hacer una busqueda por personas

busquedas_f=busqueda.busqueda.unique()
sub_area_f=sub_area.busqueda.unique()

# crea la base de trabajos buscados, esta base se realiza con base en los trabajos que los usuarios estás buscando

def compilador(data):
    vaca_df=pd.DataFrame()
    j=0
    for i in data:
        one=info_trabajo(buscador(i))
        one["trabajo_buscado"]=i
        j=j+1
        print(j)
        vaca_df=pd.concat([vaca_df,one],ignore_index=True)
    return vaca_df

vacantes_df_2=compilador(sub_area_f)
vacantes_df_1=compilador(busquedas_f)



## Fase 2: Construcción y ejecución del modelo

En esta segunda parte vas a construir el sistema de generacion de recomendaciones dados los dos dataframes: `personas_df` y `vacantes_df`. 

1. Como primera iteración podemos pensar en que una vacante es relevante para una persona si el titulo de la vacante es cercano a alguno de los trabajos incluidos en el campo `puestos_postulandose` de la persona en cuestion. Esta cercanía se puede calcular por medio de un algoritmo de match semántico con un embedding o por medio de una distancia levenstein entre cadenas de texto. 

* [¿Qué es la distancia de Levenstein?](https://es.wikipedia.org/wiki/Distancia_de_Levenshtein)
* [¿Que es el match semántico?](https://tfhub.dev/google/universal-sentence-encoder/1)

2. Si podemos extraer el nivel de ingles de las vacantes podemos hacer un filtro para que solo se le recomiende a la persona vacantes para las cuales su nivel de ingles es suficiente. 

Con base en las dos ideas anteriores, construye un algoritmo que permita recomendarle a cada persona las 5 vacantes que son mas afines según las variables `puestos_postulandose` y `nivel_de_ingles`. 

Puedes revisar el compendio general de tecnicas para similitud semántica en este [articulo](https://medium.com/@adriensieg/text-similarities-da019229c894) 


Como armarías las recomendaciones con todas las ideas y sugerencias presentadas ? deja tu codigo listo para ejecucion en el siguiente bloque: 

In [ ]:
# funcion para obtener la distancia de levenshtein y coseno 

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )          
    return (1-(matrix[size_x - 1, size_y - 1]/max(len(seq1),len(seq2))))

def coseno(data):
    list=[]
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.feature_extraction.text import CountVectorizer
    for i,j in zip(data.trabajo,data.trabajo_buscado):
        palabras=[i.lower(),j.lower().replace('"',"")]
        cuenta_vector = CountVectorizer(stop_words='spanish')
        cuenta_vector = CountVectorizer()
        matrix = cuenta_vector.fit_transform(palabras)
        matrix_fin = matrix.todense()
        df = pd.DataFrame(matrix_fin, 
                  columns=cuenta_vector.get_feature_names(), 
                  index=['palabra1', 'palabra2'])
        list.append(cosine_similarity(df, df)[1,0])
    data["dist_cosine"]=list   
    return(data)

# una funcional adicional para obeterne la distancia
def distancia(data):
    distan=[]
    for i,y in zip(data.trabajo,data.trabajo_buscado):
        dist=levenshtein(i.lower(),y.lower().replace('"',""))
        distan.append(dist)
    data["distancia"]=distan
    return(data)

# bases finales para el algortimo

dist_1=distancia(vacantes_df_1)
dist_2=distancia(vacantes_df_2)

dist_1=coseno(dist_1)
dist_2=coseno(dist_2)

a=busqueda.merge(dist_1,left_on="busqueda",right_on="trabajo_buscado",how="left")
b=sub_area.merge(dist_2,left_on="busqueda",right_on="trabajo_buscado",how="left")

a["ponderada"]=(a["distancia"]+a["dist_cosine"])/2
b["ponderada"]=(b["distancia"]+b["dist_cosine"])/2

# funcion para determinar si hay dexripcion del nivel de inlges

def nivel_ingles(data):
    ingles=[]
    for i in data.caracteristicas:
        try:
            if "ingles" in i:
                ingles.append(i)
            else: ingles.append("no")    
        except:ingles.append("no") 
    data["filtro_ingles"]=ingles
    return(data)

# Recomendacion

def recomendacion(data1=a,data2=b):
    ingles_a=nivel_ingles(data1)
    ingles_b=nivel_ingles(data2)
    ingles=pd.concat([ingles_a,ingles_b],axis=0)
    filtro_comp=[]
    if ingles[ingles["filtro_ingles"] !="no"].shape[0] !=0:
        ingles=ingles[ingles["filtro_ingles"] !="no"]
        for i,j in zip(inlges.filtro_ingles,ingles.ingles):
            if  j=="nan":
                filtro_comp.append("no")
            else: 
                if j in i:
                    filtro_comp.append("yes")
        ingles=ingles["filtro_com"]=  filtro_comp
        ingles=ingles[["filtro_com"]=="yes"]
        data=ingles.sort_values('ponderada',ascending = False).groupby('usuario').head(5)   
    else:
        data=ingles.sort_values('ponderada',ascending = False).groupby('usuario').head(5)
    return(data) 

recomendacion(a,b)

## Hazlo mejor por ti mismo !

¿Cómo podrías generar mejores recomendaciones? ¿que técnicas usarias para procesar variables de texto ? Danos tu mejor idea e inspiracion en esta parte final ideando un sistema de recomendaciones de vacantes espectacular, que sea mejor a todo lo presentado anteriormente. 

In [ ]:
# primero agregaria otras caracteristicas a los datos, como habilidades del candidato, tal ves y esto puede ser loco 
# pero agregaria tal vez pruebas para verificar conocimientos tecnicos, las vacantes usualmente piden cosas muy precisas
# por lo que puntuciones de pruebas pueden ser de gran ayuda y también con la ayuda de las empresas pues son quienes fialmente 
# saben que buscan puntualmente en las vacantes.

# frente a el modelado primero trataría de crear diccionarios donde pudiera generar sinonimos de palabras desde su significado, muchas veces el nombre de los cargos 
# los nombre de la vacantes no dicen mucho de las mismas, por lo tanto la descripción del cargo  toma importancia para entender si
# por ejemplo muchas veces los cargos de la empresas no son equivalentes dentro de los cargo que uno busca, un cargo en reporting 
# puede significar un analista de datos o un cientifico de datos o simplmente alguien operativo con concocimientos en querys
# por lo que la descripción del cargo más la deficinicón de los diferentes puesto puede lograr un macth mucho más aproximado

# finalmente dado que las empresas ya tienen un esquema o candidato objetivo despues de hacer un match de texto aplicar claificación para determinar
# de entrada cual puede ser la probabilidad de seleeción de ese usuario en un puesto X, con esto no solo se le recomendaran los cargos
# a los que el aplico sino que además se tendrpa de alguna forma una probabilidad real para el usuario de ajustarse a la vancante
# con lo que el usuario puede tener una mejor decisición sobre que es lo mejor para el

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ae6eb7be-f8e0-4cb7-81fd-2db19d09c955' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>